### Import Libraries


In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
import joblib

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

### Import Data

In [2]:
train = pd.read_csv('../data/readmissions_train.csv')
train.drop(['diag_1_desc','diag_2_desc','diag_3_desc'],axis=1,inplace=True)

X = train.drop('readmitted',axis=1)
y = train.pop('readmitted')

### Function for Keras Model

In [3]:
#First I define my keras model
def create_model():
    optimizer='adagrad'
    kernel_initializer='glorot_uniform'
    dropout=0.2
    
    model = Sequential()
    model.add(Dense(64,activation='relu',kernel_initializer=kernel_initializer))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))

    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    return model

### Define Preprocessing step per type of column

In [4]:

#Preprocessing for numerical features
numeric_features = list(set(X.columns) - set(list(X.select_dtypes('object').columns)))
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#Preprocessing for categorical features
categorical_features = list(train.select_dtypes('object').columns)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

#Preprocessor with all of the steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

### Fit the Preprocessing Pipeline

In [5]:
# Full preprocessing pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

#Train the model-Pipeline
pipeline.fit(X,y)

#Preprocess x
preprocessed = pipeline.transform(X)

### Train the Keras Classifier

In [6]:
#Create the keras model
clf = KerasClassifier(build_fn=create_model, verbose=0)
clf.fit(preprocessed,y)

### Save Custom Model files

In [7]:
joblib.dump(pipeline,'custom_model/preprocessing.pkl')
clf.model.save('custom_model/model.h5')

### Verify custom model integrity with DRUM

In [8]:
!drum validation --code-dir ./custom_model --input ../data/readmissions_test.csv --target-type binary --positive-class-label yes --negative-class-label no

2020-11-30 00:34:31.323305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:34:31.351700: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc38a8df960 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:34:31.351745: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:34:32,486 WARNING tensorflow:  From

2020-11-30 00:35:01.305383: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:35:01.325322: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc536fedbe0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:35:01.325370: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:35:02,652 WARNING tensorflow:  From

2020-11-30 00:35:34.003642: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:35:34.020405: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7f7125ee10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:35:34.020441: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:35:35,056 WARNING tensorflow:  From

2020-11-30 00:36:04.056241: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:36:04.070105: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdb9ae84980 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:36:04.070141: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:36:04,932 WARNING tensorflow:  From

2020-11-30 00:36:30.582978: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:36:30.597179: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fee013f8b90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:36:30.597214: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:36:31,495 WARNING tensorflow:  From

2020-11-30 00:36:57.936498: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:36:57.951568: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb72cf88c00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:36:57.951639: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:36:58,912 WARNING tensorflow:  From

2020-11-30 00:37:26.909208: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:37:26.928065: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd780a9a080 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:37:26.928109: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:37:27,788 WARNING tensorflow:  From

2020-11-30 00:38:02.849754: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:38:02.911358: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9f70712670 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:38:02.911674: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:38:04,124 WARNING tensorflow:  From

2020-11-30 00:38:34.358429: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:38:34.374377: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc95e7d9f40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:38:34.374418: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:38:35,263 WARNING tensorflow:  From

2020-11-30 00:38:59.902483: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-30 00:38:59.918519: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe28c87b9b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-30 00:38:59.918565: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
/Users/thodoris.petropoulos/opt/anaconda3/envs/readmissions_custom/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
2020-11-30 00:39:00,790 WARNING tensorflow:  From